In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
from scipy.stats import skew
import seaborn as sns
import sklearn
import warnings
from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge

In [3]:
train_clean = pd.read_csv('train_clean.csv', dtype={'MSSubClass': str})  
train = pd.read_csv('train.csv', dtype={'MSSubClass': str})

#### This notebook follows the stacking approach explained [here](http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/).

Essentially, we create a handful of models and use their predictions as inputs to a final model which creates our final predictions. The tricky part is avoiding leakage.  

Remove outlier records we previously identified as being harmful to the model

In [41]:
train_clean = train_clean[(train_clean.Id != 1299) & (train_clean.Id != 524)]

In [42]:
div = len(train_clean) // 5
train_clean['fold'] = 999
train_clean['m1'] = 0
train_clean['m2'] = 0
train_shuf = train_clean.sample(frac=1).copy()


for i in range(1, 6):
    start = div*i - div
    end = div*i
    train_shuf.iloc[start:end,:]['fold']= i
train_shuf.iloc[end:]['fold'] = 5    

/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [43]:
print(train_shuf.fold.head())
train_shuf.fold.tail()

689     1
696     1
770     1
1034    1
524     1
Name: fold, dtype: int64


947    5
92     5
672    5
309    5
701    5
Name: fold, dtype: int64

In [44]:
test_folds = []
for i in train_shuf.fold.unique():
    test = train_shuf[train_shuf.fold == i]
    tr = train_clean[train_shuf.fold != i]
    X_tr =  tr.drop(['SalePrice', 'Id', 'fold'], axis=1)
    X_test =  test.drop(['SalePrice', 'Id', 'fold'], axis=1)
    y_tr = tr['SalePrice']    
    
    ls = Lasso(alpha=0.001)
    ls.fit(X_tr, y_tr)
    test['m1'] = ls.predict(X_test)
    
    params = {'eta':0.01, 'max_depth':4, 'min_child_weight':4, 'subsample':0.4, 'colsample_bytree':0.8, 'scale_pos_weight':1}
    dtr = xgb.DMatrix(X_tr, y_tr)
    dtest = xgb.DMatrix(X_test)
    num_rounds = 1334
    bst = xgb.train(params, dtr, num_rounds)
    test['m2'] = bst.predict(dtest)
    
    knn = KNeighborsRegressor(n_neighbors=8)
    knn.fit(X_tr, y_tr)
    test['m3'] = knn.predict(X_test)
    
                    
                    
    rf= RandomForestRegressor(max_depth=9, n_estimators=45)
    rf.fit(X_tr, y_tr)
    test['m4'] = rf.predict(X_test)

    hb= HuberRegressor(epsilon=1.25)
    hb.fit(X_tr, y_tr)
    test['m5'] = hb.predict(X_test)

    r= Ridge(alpha=15)
    r.fit(X_tr, y_tr)
    test['m6'] = r.predict(X_test)    
    
    test_folds.append(test)

/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to b

In [45]:
tr_meta = pd.concat(test_folds)

In [46]:
test_set = pd.read_csv('test_clean.csv', dtype={'MSSubClass': str})
ids = test_set.Id.values
test_set = test_set.drop('Id', axis=1)


In [47]:
X_tr =  train_clean.drop(['SalePrice', 'm1', 'm2', 'fold', 'Id'], axis=1)
y_tr = train_clean['SalePrice']    
ls = Lasso(alpha=0.001)
ls.fit(X_tr, y_tr)


knn = KNeighborsRegressor(n_neighbors=8)
knn.fit(X_tr, y_tr)
                    
rf= RandomForestRegressor(max_depth=9, n_estimators=45)
rf.fit(X_tr, y_tr)
hb= HuberRegressor(epsilon=1.25)
hb.fit(X_tr,y_tr)

r = Ridge(alpha=15)
r.fit(X_tr, y_tr)



Ridge(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [48]:
params = {'eta':0.01, 'max_depth':4, 'min_child_weight':4, 'subsample':0.4, 'colsample_bytree':0.8, 'scale_pos_weight':1}
dtr = xgb.DMatrix(X_tr, y_tr)
dtest = xgb.DMatrix(test_set)
num_rounds = 1334
bst = xgb.train(params, dtr, num_rounds)

m1 = ls.predict(test_set)
m2 = bst.predict(dtest)
m3 = knn.predict(test_set)
m4 = rf.predict(test_set)
m5 = hb.predict(test_set)
m6 = r.predict(test_set)



test_set['m1'] = m1
test_set['m2'] = m2
test_set['m3'] = m3
test_set['m4'] = m4
test_set['m5'] = m5
test_set['m6'] = m6


In [49]:
scores = []
for i in tr_meta.fold.unique():
    test = tr_meta[tr_meta.fold == i]
    tr = tr_meta[tr_meta.fold != i]
    X_tr =  tr[['m1',  'm2', 'm3', 'm4', 'm5', 'm6', 'OverallQual']]
    X_test =  test[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'OverallQual']]
    y_test = test['SalePrice']
    y_tr = tr['SalePrice']    
    
    sv = SVR(kernel='linear', C=.5)
    sv.fit(X_tr, y_tr)
    preds = sv.predict(X_test)
    score = np.sqrt(np.mean((y_test.values - preds) **2))
    print(score)
    scores.append(score)
print('mean score: {}'.format(np.mean(scores)))

0.111322762414
0.112615828349
0.103785831089
0.1066475059
0.123609800639
mean score: 0.11159634567817882


In [50]:
scores = []
for i in tr_meta.fold.unique():
    test = tr_meta[tr_meta.fold == i]
    tr = tr_meta[tr_meta.fold != i]
    X_tr =  tr[['m1',  'm2', 'm3', 'm4', 'm5', 'm6', 'OverallQual']]
    X_test =  test[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'OverallQual']]
    y_test = test['SalePrice']
    y_tr = tr['SalePrice']    
    
    sv = SVR(kernel='linear', C=.4)
    sv.fit(X_tr, y_tr)
    preds = sv.predict(X_test)
    score = np.sqrt(np.mean((y_test.values - preds) **2))
    print(score)
    scores.append(score)
print('mean score: {}'.format(np.mean(scores)))

0.11078811412
0.112566417104
0.104113967904
0.106564008659
0.123628907772
mean score: 0.11153228311200511


In [51]:
test_set

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa,m1,m2,m3,m4,m5,m6
0,-0.635877,-0.863610,-0.112774,-1.045801,0.645902,2.287158,-0.818929,-0.251544,-0.012359,-0.427545,...,0,1,1,0,11.704515,11.759861,11.804009,11.746525,11.711204,11.684129
1,0.585094,-0.863610,-0.112774,0.169927,0.873354,-0.362565,-0.818929,-0.251544,0.205237,-0.427545,...,0,1,1,0,11.933021,11.984430,11.829794,11.926508,11.909370,11.918219
2,-0.527218,1.136698,-0.112774,0.169927,0.821649,-0.362565,-0.818929,-0.251544,-0.373436,-0.427545,...,0,1,1,0,12.107182,12.105458,12.156579,12.048355,12.111906,12.126022
3,-0.533898,1.126531,-0.112774,0.169927,0.730198,-0.362565,-0.818929,-0.251544,0.084860,-0.427545,...,0,1,1,0,12.206209,12.137350,12.215960,12.099667,12.236518,12.191520
4,0.468732,-0.863610,-0.112774,-1.045801,0.453151,-0.362565,-0.818929,-0.251544,0.695747,-0.427545,...,0,1,1,0,12.181544,12.173636,12.087263,12.197157,12.187725,12.176629
5,-1.133369,1.210147,-0.112774,0.169927,-1.417138,-0.362565,-0.818929,-0.251544,0.542178,-0.427545,...,0,1,1,0,12.045630,12.070193,12.086249,12.119573,12.026309,12.016694
6,0.235095,-0.863610,-0.112774,0.169927,0.877682,-0.362565,1.087334,-0.251544,-0.090901,-0.427545,...,0,1,1,0,12.100917,12.063315,12.041742,11.999078,12.087523,12.067201
7,-1.029631,1.125631,-0.112774,0.169927,-1.417138,-0.362565,-0.818929,-0.251544,0.560083,-0.427545,...,0,1,1,0,11.978030,12.005981,12.089894,12.080007,12.026238,11.977885
8,0.612938,-0.863610,-0.112774,-1.045801,0.749123,-0.362565,1.087334,-0.251544,0.467120,-0.427545,...,0,1,1,0,12.212134,12.150404,12.089523,12.129355,12.199087,12.165216
9,-0.684640,-0.863610,-0.112774,-1.045801,0.827110,1.963826,1.087334,-0.251544,-3.009689,-0.427545,...,0,1,1,0,11.641107,11.737365,11.741763,11.739408,11.655303,11.671792


In [52]:
X_tr =  tr_meta[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'OverallQual']]
X_test =  test_set[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'OverallQual']]
y_tr = tr_meta['SalePrice']
sv = SVR(kernel='linear', C=.4)
sv.fit(X_tr, y_tr)    

SVR(C=0.4, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [53]:
preds = np.expm1(sv.predict(X_test))

In [56]:
solution = pd.DataFrame({"id":ids, "SalePrice":preds}, columns=['id', 'SalePrice'])
solution.to_csv("stack_sv.csv", index = False)

Comparing our solution to a previous one as a sanity check.

In [57]:
pd.read_csv('stack_mult.csv').head()

,id,SalePrice
0,1461,123935.827919
1,1462,154055.408300
2,1463,184739.131214
3,1464,196656.755681
4,1465,191362.929464


In [58]:
solution.head()

,id,SalePrice
0,1461,123329.022962
1,1462,153769.137581
2,1463,185714.490223
3,1464,197115.566085
4,1465,192859.552901
